***Тема курсового проекта: "Спрогнозировать количество к заказу с помощью модели линейной регрессии y_hat = b0 + b1*x1 + b2*x2 + ... bn*xn матричным способом"***

**Описание датасета**

* **НомерЗаказа**     - номер документа Автозаказ в программе 1с
* **ГодЗаказа**       - год из поля дата документа Автозаказ
* **МесяцЗаказа**     - месяц из поля дата документа Автозаказ
* **ДеньЗаказа**      - день из поля дата документа Автозаказ
* **НеделяЗаказа**    - номер недели из поля дата документа Автозаказ
* **КодТовара**       - код товара, количество которого необходимо заказать (из табличной части документа Автозаказ)
* **ЦЕНА_ЗКП_РУБ**    - цена закупочная в рублях
* **СРОК_ГДН_ОСТ_Д**  - срок годности остаточный, в днях
* **ОСТ_ТЕК_ШТ**      - текущий остаток в шт/кг
* **ТОВ_В_ПУТ_СУМ_ШТ**- товара в пути в шт/кг
* **ПРДЖ_СР_ШТ_ДО**   - продажи средние шт/кг
* **КВО_ПРДЖ_БАЗА_ШТ**- продажи базовые в шт
* **ПРДЖ_БАЗА_СР_ШТ** - продажи, базовые, средние, шт/кг
* **ППП_ДЛН_Д**       - период покрытия продаж длительность дней
* **ТРБ_МАГ_КВНТ_ШТ** - требование магазина, квант, шт/кг
* **ПРДЖ_ШТ_НЕД**     - недельные продажи 
* **СПИС_ШТ_НЕД**     - недельные списания 

* **ЗКЗ_ИТГ_ШТ**    - количество к заказу, наша целевая переменная, которую мы будем пытаться предсказать

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [13]:
df = pd.read_csv('Order1c_pred.csv')
df.head()

,НомерЗаказа,ГодЗаказа,МесяцЗаказа,ДеньЗаказа,НеделяЗаказа,КодТовара,ЦЕНА_ЗКП_РУБ,СРОК_ГДН_ОСТ_Д,ОСТ_ТЕК_ШТ,ТОВ_В_ПУТ_СУМ_ШТ,ПРДЖ_СР_ШТ_ДО,КВО_ПРДЖ_БАЗА_ШТ,ПРДЖ_БАЗА_СР_ШТ,ППП_ДЛН_Д,ТРБ_МАГ_КВНТ_ШТ,ПРДЖ_ШТ_НЕД,СПИС_ШТ_НЕД,ЗКЗ_ИТГ_ШТ
0,M3300009893,2018,4,24,17,389 965,67.85,245,21.0,0.0,0.68421,3,5.50612,14,6.0,2.0,0.0,6.0
1,M3300009893,2018,4,24,17,389 970,36.03,245,14.0,0.0,0.45714,5,3.28168,14,12.0,3.0,0.0,12.0
2,M3300009893,2018,4,24,17,366 976,30.54,482,12.0,0.0,0.31428,5,2.20001,14,6.0,0.0,0.0,6.0
3,M3300009893,2018,4,24,17,367 448,46.40,241,11.0,0.0,0.10345,4,0.79365,14,10.0,0.0,1.0,10.0
4,M3300009893,2018,4,24,17,260 461,50.92,245,12.0,0.0,0.39999,5,2.62693,14,12.0,4.0,0.0,12.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659803 entries, 0 to 659802
Data columns (total 18 columns):
НомерЗаказа         659803 non-null object
ГодЗаказа           659803 non-null int64
МесяцЗаказа         659803 non-null int64
ДеньЗаказа          659803 non-null int64
НеделяЗаказа        659803 non-null int64
КодТовара           659803 non-null object
ЦЕНА_ЗКП_РУБ        659803 non-null float64
СРОК_ГДН_ОСТ_Д      659803 non-null int64
ОСТ_ТЕК_ШТ          659803 non-null float64
ТОВ_В_ПУТ_СУМ_ШТ    659803 non-null float64
ПРДЖ_СР_ШТ_ДО       659803 non-null float64
КВО_ПРДЖ_БАЗА_ШТ    659803 non-null int64
ПРДЖ_БАЗА_СР_ШТ     659803 non-null float64
ППП_ДЛН_Д           659803 non-null int64
ТРБ_МАГ_КВНТ_ШТ     659803 non-null float64
ПРДЖ_ШТ_НЕД         659803 non-null float64
СПИС_ШТ_НЕД         659803 non-null float64
ЗКЗ_ИТГ_ШТ          659803 non-null float64
dtypes: float64(9), int64(7), object(2)
memory usage: 90.6+ MB


In [19]:
df1 = df.select_dtypes(include = ['float64', 'int64'])
df1.head()

,ГодЗаказа,МесяцЗаказа,ДеньЗаказа,НеделяЗаказа,ЦЕНА_ЗКП_РУБ,СРОК_ГДН_ОСТ_Д,ОСТ_ТЕК_ШТ,ТОВ_В_ПУТ_СУМ_ШТ,ПРДЖ_СР_ШТ_ДО,КВО_ПРДЖ_БАЗА_ШТ,ПРДЖ_БАЗА_СР_ШТ,ППП_ДЛН_Д,ТРБ_МАГ_КВНТ_ШТ,ПРДЖ_ШТ_НЕД,СПИС_ШТ_НЕД,ЗКЗ_ИТГ_ШТ
0,2018,4,24,17,67.85,245,21.0,0.0,0.68421,3,5.50612,14,6.0,2.0,0.0,6.0
1,2018,4,24,17,36.03,245,14.0,0.0,0.45714,5,3.28168,14,12.0,3.0,0.0,12.0
2,2018,4,24,17,30.54,482,12.0,0.0,0.31428,5,2.20001,14,6.0,0.0,0.0,6.0
3,2018,4,24,17,46.40,241,11.0,0.0,0.10345,4,0.79365,14,10.0,0.0,1.0,10.0
4,2018,4,24,17,50.92,245,12.0,0.0,0.39999,5,2.62693,14,12.0,4.0,0.0,12.0


In [30]:
y = df1.iloc[:,-1].values
y

array([ 6., 12.,  6., ...,  6.,  2.,  9.])

In [27]:
y.shape

(659803,)

In [28]:
y = y.reshape(659803, 1)
y

array([[ 6.],
       [12.],
       [ 6.],
       ...,
       [ 6.],
       [ 2.],
       [ 9.]])

In [29]:
y.shape

(659803, 1)

In [33]:
X = df1.iloc[:,0:-1].values
X

array([[2.018e+03, 4.000e+00, 2.400e+01, ..., 6.000e+00, 2.000e+00,
        0.000e+00],
       [2.018e+03, 4.000e+00, 2.400e+01, ..., 1.200e+01, 3.000e+00,
        0.000e+00],
       [2.018e+03, 4.000e+00, 2.400e+01, ..., 6.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.017e+03, 1.000e+01, 2.700e+01, ..., 3.000e+00, 2.966e+00,
        0.000e+00],
       [2.017e+03, 1.000e+01, 2.700e+01, ..., 2.000e+00, 1.018e+00,
        0.000e+00],
       [2.017e+03, 1.000e+01, 2.700e+01, ..., 9.000e+00, 8.000e+00,
        0.000e+00]])

In [32]:
X.shape

(659803, 14)

In [34]:
X = np.hstack([np.ones((659803, 1)), X])
X

array([[1.000e+00, 2.018e+03, 4.000e+00, ..., 6.000e+00, 2.000e+00,
        0.000e+00],
       [1.000e+00, 2.018e+03, 4.000e+00, ..., 1.200e+01, 3.000e+00,
        0.000e+00],
       [1.000e+00, 2.018e+03, 4.000e+00, ..., 6.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.000e+00, 2.017e+03, 1.000e+01, ..., 3.000e+00, 2.966e+00,
        0.000e+00],
       [1.000e+00, 2.017e+03, 1.000e+01, ..., 2.000e+00, 1.018e+00,
        0.000e+00],
       [1.000e+00, 2.017e+03, 1.000e+01, ..., 9.000e+00, 8.000e+00,
        0.000e+00]])

In [35]:
X.shape

(659803, 16)

In [36]:
B = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T@y)
B

array([-2.13655271e+02,  1.05755640e-01, -1.32233294e+00, -4.12524605e-02,
        3.03691840e-01,  1.06169118e-03,  2.52238678e-07,  8.01573847e-02,
       -6.89438738e-02,  7.23390384e-01, -2.19122243e-01,  2.33638361e-01,
        4.03145838e-01,  1.03879572e+00,  1.53487235e-02, -7.01493251e-02])

In [37]:
B.shape

(16,)

In [39]:
np.sum(B*X)

4855130.398902943

In [41]:
C = B*X
C

array([[-2.13655271e+02,  2.13414882e+02, -5.28933177e+00, ...,
         6.23277435e+00,  3.06974469e-02, -0.00000000e+00],
       [-2.13655271e+02,  2.13414882e+02, -5.28933177e+00, ...,
         1.24655487e+01,  4.60461704e-02, -0.00000000e+00],
       [-2.13655271e+02,  2.13414882e+02, -5.28933177e+00, ...,
         6.23277435e+00,  0.00000000e+00, -0.00000000e+00],
       ...,
       [-2.13655271e+02,  2.13309126e+02, -1.32233294e+01, ...,
         3.11638717e+00,  4.55243138e-02, -0.00000000e+00],
       [-2.13655271e+02,  2.13309126e+02, -1.32233294e+01, ...,
         2.07759145e+00,  1.56250005e-02, -0.00000000e+00],
       [-2.13655271e+02,  2.13309126e+02, -1.32233294e+01, ...,
         9.34916152e+00,  1.22789788e-01, -0.00000000e+00]])

In [42]:
C.shape

(659803, 16)

In [43]:
np.dot?

In [45]:
mse = ((y-B.T*X)**2).sum()/10
mse

ValueError: operands could not be broadcast together with shapes (659803,) (659803,16) 